# Description
This script can be used for an optimization with an cylinder as an initial geometry. The diameter from the cylinder is fixed.

# General Form of the truncated cone

In [1]:
def sech(x):
    return 1 / np.cosh(x)

In [2]:
import copy
#from ._curvatures import *

from ddgclib._curvaturesplay import *

def truncated_cone(u, v, r_l, r_u, length,v_l = 0,shift_activated = False, x_shift = 0, y_shift=0, z_shift = 0, rotation=False, rotation_angle=45, shearing = False, shearing_parameter = 1):
    '''
    u =
    v =
    r_l = radian of the initial truncated cone on the lower site
    r_u = radian of the initial truncated cone on the upper site
    length = length of the liquid bridge
    v_l = Starting position of the truncated cone i the z-direction
    rotation = should the Complex be rotated
        Selection-Options:
            True: The complex will be rotated around the y-Axis
            False: The complex is lying on the z-axis

    rotation_angle = should the Complex be sheared in the z-direction (maybye relevant for tangential movement)
        Selection-Options:
            True: The complex will be sheared into the positive z-direction
            False: The complex is lying on the z-axis
    shearing =
    shearing_parameter =

    '''
    v_u = length
    r = r_l + (r_u - r_l) * v /(v_u - v_l)

    # Parametrisierung Zylinder
    x = r * np.cos(u)
    y = r * np.sin(u)
    z = v

    if 0:
        x += x_shift
        y += y_shift
        z += z_shift

    if shift_activated:
        x += x_shift
        y += y_shift
        z += z_shift

    if rotation:
        x1 = x
        y1 = y
        z1 = z

        x = x1
        y = np.cos(rotation_angle* np.pi/180.0) *y1 - np.sin(rotation_angle* np.pi/180.0)*z1
        z = np.sin(rotation_angle* np.pi/180.0) *y1 + np.cos(rotation_angle* np.pi/180.0) *z1



    if shearing:
        x1 = x
        y1 = y
        z1 = z

        x = x1
        y = z1 * shearing_parameter + y1
        z = z1

    return x,y,z

In [3]:

#from ddgclib._plotting import *

def truncated_cone_initial_N(r_l, r_u, length, refinement,v_l = 0,shift_activated = False, x_shift = 0, y_shift=0, z_shift = 0,rotation=False,rotation_angle=45, shearing = False, shearing_parameter = 1):
    '''
    r_l = radian of the initial truncated cone on the lower site
    r_u = radian of the initial truncated cone on the upper site
    length = length of the liquid bridge
    refinement = refinement
    v_l = Starting position of the truncated cone i the z-direction

    '''

    v_u = length + v_l
    a = 1

    domain = [#(-2.0, 2.0),  # u
              #(0.0, 2 * np.pi),  # u
              (0.0, 2 * np.pi),  # u
              (v_l, v_u)  # v
              ]

    HC_plane = Complex(2, domain)
    HC_plane.triangulate()
    for i in range(refinement):
        HC_plane.refine_all()

    # H
    HC = Complex(3, domain)
    bV = set()
    cdist = 1e-8

    u_list = []
    v_list = []
    for v in HC_plane.V:
        x, y, z = truncated_cone(*v.x_a,r_l,r_u,length,shift_activated = shift_activated, x_shift = x_shift, y_shift=y_shift, z_shift = z_shift,rotation=rotation,rotation_angle=rotation_angle, shearing=shearing, shearing_parameter=shearing_parameter)
        v2 = HC.V[tuple([x, y, z])]

        u_list.append(v.x_a[0])
        v_list.append(v.x_a[1])

        #TODO: Does not work at all:
        boundary_bool = (
                        v.x[1] == domain[1][0] or v.x[1] == domain[1][1]
                        #v.x[2] == domain[0][0] or v.x[2] == domain[0][1]
                       # or v.x[1] == domain[1][0] or v.x[1] == domain[1][1]
                         )
        if boundary_bool:
            bV.add(v2)

    # Connect neighbours
    for v in HC_plane.V:
        for vn in v.nn:
            v1 = list(HC.V)[v.index]
            v2 = list(HC.V)[vn.index]
            v1.connect(v2)


    # Remerge:
    HC.V.merge_all(cdist=cdist)
    bVc = copy.copy(bV)
    for v in bVc:
        #print(f'bv = {v.x}')
        if not (v in HC.V):
            bV.remove(v)

    H_f = []
    bV_H_f = []
    K_f = []
    bV_K_f = []

    neck_verts = []
    neck_sols = []
    for vert in HC.V:
        if not (vert in bV):
            for u_i, v_i in zip(u_list, v_list):
                #catenoid(u, v,r_l,r_u,length)
                #x, y, z = truncated_cone(*v.x_a,r_l,r_u,length,rotation=rotation,rotation_angle=rotation_angle, shearing=shearing, shearing_parameter=shearing_parameter)
                x, y, z = truncated_cone(u_i, v_i,r_l,r_u,length,shift_activated = shift_activated, x_shift = x_shift, y_shift=y_shift, z_shift = z_shift,rotation=rotation,rotation_angle=rotation_angle, shearing=shearing, shearing_parameter=shearing_parameter)
                va = np.array([x, y, z])
                #print(f'va == vert.x_a = {va == vert.x_a}')
                ba = va == vert.x_a
                if ba.all():
                    #print(f'ba.all() = {ba.all()}')
                    u = u_i
                    v = v_i
                    H_f_i = 0.0
                    K_f_i = -(sech(v/a))**4 / (a**2)
                    H_f.append(H_f_i)
                    K_f.append(K_f_i)
                    #TODO: MERGE NEAR VERTICES

                    if z == 0.0:
                        neck_verts.append(vert.index)
                        neck_sols.append((H_f_i, K_f_i))


    return HC, bV, K_f, H_f, neck_verts, neck_sols





In [5]:
import polyscope as ps
from ddgclib._misc import *  # coldict is neeeded

def plot_polyscope(HC):
    # Initialize polyscope
    ps.init()
    ps.remove_all_structures()
    ps.set_up_dir("z_up")

    do = coldict['db']
    lo = coldict['lb']
    HC.dim = 2  # The dimension has changed to 2 (boundary surface)
    HC.vertex_face_mesh()
    points = np.array(HC.vertices_fm)
    triangles = np.array(HC.simplices_fm_i)
    ### Register a point cloud
    # `my_points` is a Nx3 numpy array
    my_points = points
    ps_cloud = ps.register_point_cloud("my points", my_points)
    ps_cloud.set_color(tuple(do))
    #ps_cloud.set_color((0.0, 0.0, 0.0))
    verts = my_points
    faces = triangles
    ### Register a mesh
    # `verts` is a Nx3 numpy array of vertex positions
    # `faces` is a Fx3 array of indices, or a nested list
    surface = ps.register_surface_mesh("my mesh", verts, faces,
                             color=do,
                             edge_width=1.0,
                             edge_color=(0.0, 0.0, 0.0),
                             smooth_shade=False)

    # Add a scalar function and a vector function defined on the mesh
    # vertex_scalar is a length V numpy array of values
    # face_vectors is an Fx3 array of vectors per face
    if 0:
        ps.get_surface_mesh("my mesh").add_scalar_quantity("my_scalar",
                vertex_scalar, defined_on='vertices', cmap='blues')
        ps.get_surface_mesh("my mesh").add_vector_quantity("my_vector",
                face_vectors, defined_on='faces', color=(0.2, 0.5, 0.5))

    # View the point cloud and mesh we just registered in the 3D UI
    #ps.show()
    ps.show()



In [6]:
import polyscope as ps
from ddgclib._misc import *  # coldict is neeeded
import time

def plot_polyscope2(HC,HC2,HC3):
    # Initialize polyscope

    ps.init()
    ps.remove_all_structures()
    ps.set_up_dir("z_up")

    do = coldict['db']
    lo = coldict['lb']

    HC.dim = 2  # The dimension has changed to 2 (boundary surface)
    HC2.dim = 2
    HC3.dim = 2

    HC.vertex_face_mesh()
    points = np.array(HC.vertices_fm)
    triangles = np.array(HC.simplices_fm_i)

    HC2.vertex_face_mesh()
    points2 = np.array(HC2.vertices_fm)
    triangles2 = np.array(HC2.simplices_fm_i)

    HC3.vertex_face_mesh()
    points3 = np.array(HC3.vertices_fm)
    triangles3 = np.array(HC3.simplices_fm_i)
    ### Register a point cloud
    # `my_points` is a Nx3 numpy array

    my_points = points
    ps_cloud = ps.register_point_cloud("my points", my_points)
    ps_cloud.set_color(tuple(do))

    my_points2 = points2
    ps_cloud2 = ps.register_point_cloud("my points2", my_points2)
    ps_cloud2.set_color(tuple(do))

    my_points3 = points3
    ps_cloud3 = ps.register_point_cloud("my points3", my_points3)
    ps_cloud3.set_color(tuple(do))

    verts = my_points
    faces = triangles


    verts2 = my_points2
    faces2 = triangles2

    verts3 = my_points3
    faces3 = triangles3



    surface1 = ps.register_surface_mesh("my mesh", verts, faces,
                     color=do,
                     edge_width=1.0,
                     edge_color=(0.0, 0.0, 0.0),
                     smooth_shade=False)
    time.sleep(2)
    surface2 = ps.register_surface_mesh("my mesh2", verts2, faces2,
                     color=do,
                     edge_width=1.0,
                     edge_color=(0.0, 0.0, 0.0),
                     smooth_shade=False)
    time.sleep(2)
    surface3 = ps.register_surface_mesh("my mesh3", verts3, faces3,
                         color=do,
                         edge_width=1.0,
                         edge_color=(0.0, 0.0, 0.0),
                         smooth_shade=False)


    #ps_mesh.set_transparency(0.5
    surface1.set_transparency(1)
    surface2.set_transparency(1)
    surface3.set_transparency(1)

    # View the point cloud and mesh we just registered in the 3D UI
    #ps.show()
    ps.show()

In [7]:
import polyscope as ps
from ddgclib._misc import *  # coldict is neeeded
import time



def plot_polyscope3(HC,HC2,HC3,particle_j,particle_k,particle_i = {'x':0,'y':0,'z':0,'r':1}):
    '''
    HC,HC2,HC3 = Complex of the bridges, that should be plotted
    particle_j = Dictionarys with  structure
    particle_k

    '''
    Pos1 = np.array([[particle_i['x'],particle_i['y'],particle_i['z']]]) # mittel-Position -Particle i
    Pos2 = np.array([[particle_j['x'],particle_j['y'],particle_j['z']]]) # mittel-Position -Particle j
    Pos3 = np.array([[particle_k['x'],particle_k['y'],particle_k['z']]]) # mittel-Position -Particle k

    # Initialize polyscope

    ps.init()
    ps.remove_all_structures()
    ps.set_up_dir("z_up")

    do = coldict['db']
    lo = coldict['lb']

    ps_cloud1 = ps.register_point_cloud("particle_i", Pos1, point_render_mode='sphere')
    ps_cloud2 = ps.register_point_cloud("particle_j", Pos2, point_render_mode='sphere')
    ps_cloud3 = ps.register_point_cloud("particle_k", Pos3, point_render_mode='sphere')

    ps_cloud1.set_radius(particle_i['r'], relative=False)
    ps_cloud2.set_radius(particle_j['r'], relative=False)
    ps_cloud3.set_radius(particle_k['r'], relative=False)

    ps_cloud1.set_transparency(0.5)
    ps_cloud2.set_transparency(0.5)
    ps_cloud3.set_transparency(0.5)

    HC.dim = 2  # The dimension has changed to 2 (boundary surface)
    HC2.dim = 2
    HC3.dim = 2

    HC.vertex_face_mesh()
    points = np.array(HC.vertices_fm)
    triangles = np.array(HC.simplices_fm_i)

    HC2.vertex_face_mesh()
    points2 = np.array(HC2.vertices_fm)
    triangles2 = np.array(HC2.simplices_fm_i)

    HC3.vertex_face_mesh()
    points3 = np.array(HC3.vertices_fm)
    triangles3 = np.array(HC3.simplices_fm_i)
    ### Register a point cloud
    # `my_points` is a Nx3 numpy array

    my_points = points
    ps_cloud = ps.register_point_cloud("my_points", my_points)
    ps_cloud.set_color(tuple(do))

    my_points2 = points2
    ps_cloud2 = ps.register_point_cloud("my_points2", my_points2)
    ps_cloud2.set_color(tuple(do))

    my_points3 = points3
    ps_cloud3 = ps.register_point_cloud("my_points3", my_points3)
    ps_cloud3.set_color(tuple(do))

    verts = my_points
    faces = triangles


    verts2 = my_points2
    faces2 = triangles2

    verts3 = my_points3
    faces3 = triangles3



    surface1 = ps.register_surface_mesh("bridge_ij", verts, faces,
                     color=do,
                     edge_width=1.0,
                     edge_color=(0.0, 0.0, 0.0),
                     smooth_shade=False)
    #time.sleep(2)
    surface2 = ps.register_surface_mesh("bridge_ik", verts2, faces2,
                     color=do,
                     edge_width=1.0,
                     edge_color=(0.0, 0.0, 0.0),
                     smooth_shade=False)
    #time.sleep(2)
    surface3 = ps.register_surface_mesh("bridge_jk", verts3, faces3,
                         color=do,
                         edge_width=1.0,
                         edge_color=(0.0, 0.0, 0.0),
                         smooth_shade=False)


    #ps_mesh.set_transparency(0.5
    surface1.set_transparency(1)
    surface2.set_transparency(1)
    surface3.set_transparency(1)

    # View the point cloud and mesh we just registered in the 3D UI
    #ps.show()
    ps.show()

# Ausführen der Datei

In [8]:
#HC, bV, K_f, H_f, neck_verts, neck_sols = truncated_cone_initial_N(r_l, r_u, length, refinement,v_l = 0,shift_activated = False, x_shift = 0, y_shift=0, z_shift = 0,rotation=False,rotation_angle=45, shearing = False, shearing_parameter = 1)
r_l = 1
r_u = 2
#r_u = r_l
length = 1
refinement = 3

length1 = 2.5

'''
Pos1 = np.array([[0,0,0]]) # middle-Position - Particle i
Pos2 = np.array([[0,0,3]]) # middle-Position - Particle j
Pos3 = np.array([[2,2,3]]) # middle-Position - Particle k
'''

# Radian of the particle
r_i = 1
r_j = 1
r_k = 1

particle_i = {'x':0,'y':0,'z':0,'r':r_i}     # x-Coordinate, y-coordinate and z-coordinate must be 0!
particle_j = {'x':0,'y':0,'z':3,'r':r_j}      # x-Coordinate and y-coordinate must be 0!
particle_k = {'x':0,'y':3,'z':0,'r':r_k}      # x-Coordinate must be 0!

# np.arctan((particle_i['z']-particle_k['z'])/particle_k['y']) * 180/np.pi
#Angles to calculate the bridge diameter in °
angle_ij_i = 20
angle_ij_j = 30

angle_ik_i = 20
angle_ik_k = 30

angle_jk_j = 10
angle_jk_k = 10

'''
------------------- After this lines no Eingaben are necessary
'''

# Calculating the diameter of the bridges
diameter_ij_i =  np.sin(angle_ij_i * np.pi/180.0) * r_i
diameter_ij_j =  np.sin(angle_ij_j * np.pi/180.0) * r_j

diameter_ik_i = 2 * np.sin(angle_ik_i * np.pi/180.0) * r_i
diameter_ik_k = 2 * np.sin(angle_ik_k * np.pi/180.0) * r_j

diameter_jk_j = 2 * np.sin(angle_jk_j * np.pi/180.0) * r_j
diameter_jk_k = 2 * np.sin(angle_jk_k * np.pi/180.0) * r_k

# Calculating the length of the circle-Segment
h_ij_i = particle_i['r'] -  (particle_i['r']**2 - (diameter_ij_i/2)**2)**0.5
h_ij_j = particle_j['r'] -  (particle_j['r']**2 - (diameter_ij_j/2)**2)**0.5

h_ik_i = particle_i['r'] -  (particle_i['r']**2 - (diameter_ik_i/2)**2)**0.5
h_ik_k = particle_k['r'] -  (particle_k['r']**2 - (diameter_ik_k/2)**2)**0.5

h_jk_j = particle_j['r'] -  (particle_j['r']**2 - (diameter_jk_j/2)**2)**0.5
h_jk_k = particle_k['r'] -  (particle_k['r']**2 - (diameter_jk_k/2)**2)**0.5
#calculated angles of the bridges
#rotation_angle_ik = np.arctan(particle_k['z']/particle_k['y']) * 180/np.pi

if particle_k['z'] == 0:
    rotation_angle_ik = 90
else:
    rotation_angle_ik = np.arctan(particle_k['y']/particle_k['z']) * 180/np.pi

if particle_k['z'] == particle_j['z']:
    rotation_angle_jk = 90
else:
    rotation_angle_jk = np.arctan((particle_j['z']-particle_k['z'])/particle_k['y']) * 180/np.pi


rotation_angle_jk = np.arctan((particle_j['z']-particle_k['z'])/particle_k['y']) * 180/np.pi

# length of the bridges
length_ij = particle_j['z']  - (particle_i['r'] + particle_j['r']) + h_ij_j + h_ij_i
length_ik = ((particle_k['z']-particle_i['z'])**2 + (particle_k['y']-particle_i['y'])**2)**0.5 - (particle_i['r'] + particle_k['r']) + h_ik_i + h_ik_k
length_jk =  ((particle_j['z']-particle_k['z'])**2 + (particle_k['y']-particle_j['y'])**2)**0.5 -  (particle_j['r'] + particle_k['r']) + h_jk_j + h_jk_k

# rotation of the bridges
rotation_ij = 0
rotation_ik = -rotation_angle_ik# + 180
print(f'rotation_angle_ik = {rotation_angle_ik}')

#rotation_ik = 90
rotation_jk = -rotation_angle_jk
print(f'rotation_angle_jk = {rotation_angle_jk}')

rotation_jk = 0

bridge_ij = {'bridge_radian_i':diameter_ij_i/2, 'bridge_radian_j':diameter_ij_j/2, 'length':length_ij, 'rotation':rotation_ij, 'start_position': particle_i['r']- h_ij_i }

bridge_ik = {'bridge_radian_i':diameter_ik_i/2, 'bridge_radian_k':diameter_ik_k/2, 'length':length_ik, 'rotation':rotation_ik, 'angle':rotation_angle_ik,'start_position': particle_i['r']-h_ik_i}

bridge_jk = {'bridge_radian_j':diameter_jk_j/2, 'bridge_radian_k':diameter_jk_k/2, 'length':length_jk, 'rotation':rotation_jk, 'start_position': particle_j['z'] + particle_j['r'] - h_jk_j}

#bridge_ij is ready
bridge_ij_complex = truncated_cone_initial_N(bridge_ij['bridge_radian_i'], bridge_ij['bridge_radian_j'], bridge_ij['length'], refinement,v_l = bridge_ij['start_position'],shift_activated = False, x_shift = 0, y_shift=0, z_shift = - h_ij_i,rotation=False,rotation_angle=60, shearing = False, shearing_parameter = 1)

bridge_ik_complex = truncated_cone_initial_N(bridge_ik['bridge_radian_i'], bridge_ik['bridge_radian_k'], bridge_ik['length'], refinement,v_l = bridge_ik['start_position'],shift_activated = True, x_shift = 0, y_shift = 0, z_shift = 0, rotation=True,rotation_angle = bridge_ik['rotation'], shearing = False, shearing_parameter = 1)

bridge_jk_complex = truncated_cone_initial_N(bridge_jk['bridge_radian_j'], bridge_jk['bridge_radian_k'], bridge_ik['length'], refinement,v_l = 0,shift_activated = True, x_shift = particle_j['x'], y_shift=particle_j['y'], z_shift = particle_j['z'],rotation=False,rotation_angle = bridge_jk['rotation'], shearing = False, shearing_parameter = 0)

#bridge_jk_complex = truncated_cone_initial_N(bridge_jk['bridge_radian_j'], bridge_jk['bridge_radian_k'], bridge_ik['length'], refinement,v_l = 0,shift_activated = True, x_shift = 0, y_shift=0, z_shift = 0,rotation=True,rotation_angle = bridge_jk['rotation'], shearing = False, shearing_parameter = 0)

#plot_polyscope(dummy_2[0])


#particle_k = {'x':0,'y':4,'z':0.0,'r':r_k}
#plot_polyscope(dummy_1[0])

rotation_angle_ik = 90
rotation_angle_jk = 45.0


In [9]:
bridge_jk_complex[0].V
HC, bV = bridge_jk_complex[0], bridge_jk_complex[1]

In [13]:
import copy
hcv = copy.copy(HC.V)
rotation_angle = 45
for v in hcv:
    'Move the vertices to the rotated bridge'
    x1, y1, z1 = v.x[0], v.x[1], v.x[2]

    x = x1
    y = np.cos(rotation_angle* np.pi/180.0) *y1 - np.sin(rotation_angle* np.pi/180.0)*z1
    z = np.sin(rotation_angle* np.pi/180.0) *y1 + np.cos(rotation_angle* np.pi/180.0) *z1
    xa = (x, y, z)
    HC.V.move(v, xa)


In [11]:
bV

{<ddgclib._vertex.VertexCube at 0x188fff96850>,
 <ddgclib._vertex.VertexCube at 0x188fffb2370>}

In [15]:
#plot_polyscope3(bridge_ij_complex[0],bridge_ik_complex[0],bridge_jk_complex[0],particle_j,particle_k,particle_i = particle_i)

plot_polyscope(HC)

np.zeros([self.V.size(), self.dim]) = [[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]